In [1]:
# OpenAI API Key 정의
# **주의!** OpenAI API Key 는 외부에 노출하지 않도록 주의한다. 
# 4월 17일 이후에는 이전에 사용하던 사용자 키(User Key)보다 프로젝트별로 구분할 수 있는 프로젝트 키(Project Key)를 활용하는 것을 권장
# 일반적으로 시스템의 환경변수에 숨겨놓고 소스코드에는 노출하지 않지만, 
# 해당 과정은 파이썬 활용에 관한 내용이므로 예제에서는 생략한다.
# [모델 문서](https://platform.openai.com/docs/models)에는 사용가능한 모델의 이름이 나열되어있고, OpenAI에서 모델을 업데이트할 때마다 이곳에 명시하므로 자주 확인하면 좋다.

try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print(f"> 구글 코랩의 보안 비밀에서 OPENAI_API_KEY를 불러옵니다.")

except:
    OPENAI_API_KEY = "YOUR_OPENAI_PROJECT_KEY"
    print(f"> 로컬 환경에서 설정한 OPENAI_API_KEY를 사용합니다.")

OPENAI_MODEL = "gpt-4-turbo"

In [2]:
# OpenAI API 패키지 import
try:
    import openai

except ModuleNotFoundError:
    # OpenAI API 패키지 설치
    !pip install openai pandas

finally:
    import openai

if "1.23.1" != openai.VERSION:
    print(f">> 예제는 2024-04-17 update 버전을 기준으로 작성하였습니다. 현재 버전 ({openai.VERSION})")

### [Assistants Tool] File Search

이 도구는 어시스턴트가 업로드한 파일들의 내용을 참고하여 사용자의 요청에 응답할 수 있는 기능 제공

⚠️"파일 검색(File Search)" 도구는 기존의 "지식 검색(Knowledge Retrieval)" 기능을 발전시켜 2024년 4월에 새롭게 변경되었다.

In [17]:
from pathlib import Path
import pandas as pd

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

# 0. 데이터 파일 업로드 및 벡터 저장소 구성
report_files = [
    "datafiles/[LG전자]사업보고서(2024.03.18).pdf",
    "datafiles/[삼성전자]사업보고서(2024.03.12).pdf",
    "datafiles/[현대자동차]사업보고서(2024.03.13).pdf",
    "datafiles/alphabet-10-k-2023.pdf",
    "datafiles/amd-10-k-2023.pdf",
    "datafiles/apple-10-k-2023.pdf",
    "datafiles/nvidia-10-k-2023.pdf"
]

# 같은 파일을 중복 업로드하는 것을 방지하기 위해 목록에 없는 파일만 업로드한다.
filelist = [file.filename for file in client.files.list().data]

for filepath in report_files:
    if Path(filepath).name not in filelist:
        file = client.files.create(
            file=open(filepath, "rb"),
            purpose="assistants"
        )
        print(f"{filepath} : {file.id}")
    else:
        print(f"{filepath} : already uploaded.")


name_vector_store = "사업보고서 및 재무제표"

# 같은 벡터 저장소를 생성하지 않도록 벡터 저장소 목록을 확인한다.
vector_store_dict = {vs.id : vs.name for vs in client.beta.vector_stores.list()}

if name_vector_store not in vector_store_dict.values():
    file_list = client.files.list(purpose="assistants")
   
    df_files = pd.DataFrame([data.__dict__ for data in file_list.data])
    df_filtered = df_files[df_files['filename'].str.contains('사업보고서|10-k', case=False)]
    file_ids = df_filtered["id"].to_list()

    vector_store = client.beta.vector_stores.create(
        name=name_vector_store,
        file_ids=file_ids
    )
else:
    # 이미 같은 이름의 벡터 저장소가 있다면 첫 번째 벡터 저장소를 활용한다.
    print(f"{name_vector_store} : already created.")
    vsid = [key for key, value in vector_store_dict.items() if value == name_vector_store][0]
    vector_store = client.beta.vector_stores.retrieve(vector_store_id=vsid)
    print(f"{vector_store.id} : retrieved.")

datafiles/[LG전자]사업보고서(2024.03.18).pdf : file-t75xx5t1TZKEdk46s8wt8lqQ
datafiles/[삼성전자]사업보고서(2024.03.12).pdf : file-KVALuC6E6KqCiKjiIO2kCy8R
datafiles/[현대자동차]사업보고서(2024.03.13).pdf : file-c62WFWYFhwvbnPe2tlqGOcYo
datafiles/alphabet-10-k-2023.pdf : file-u6Q8WEBWWjC9Rqefn81aFOlP
datafiles/amd-10-k-2023.pdf : file-a2SzUTuLi75IlhDde8GXbrM6
datafiles/apple-10-K-2023.pdf : file-maKbbflZCGVlAfaU3zXSQd5t
datafiles/nvidia-10-K-2023.pdf : file-9smC0qKDHupcgOuKmTgxsLjB
SyncPage[FileObject](data=[FileObject(id='file-9smC0qKDHupcgOuKmTgxsLjB', bytes=3550065, created_at=1713794878, filename='nvidia-10-K-2023.pdf', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-maKbbflZCGVlAfaU3zXSQd5t', bytes=2629926, created_at=1713794872, filename='apple-10-K-2023.pdf', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-a2SzUTuLi75IlhDde8GXbrM6', bytes=3783571, created_at=1713794868, filename='amd-10-k-2023.pdf', obje

In [18]:
# 1. 스레드 생성
thread = client.beta.threads.create()

# 2. 어시스턴트 생성; 파일 검색 도구 추가 
assistant = client.beta.assistants.create(
    name="재무 분석 전문가",
    instructions="""
        당신은 재무 분석 전문가입니다. 제공한 사업보고서 및 재무제표 정보를 바탕으로 질문에 답하시오.
        """,
    model=OPENAI_MODEL,
    tools=[{"type": "file_search"}],
    tool_resources={     
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
)

tool called : file_search, call_KfzuZQ3TZXJBKEibLpUNMZCt
2023년 주력 상품은 다음과 같습니다:

1. **삼성전자**: 2023년에는 여러 제품군의 업그레이드와 발표가 있었습니다:
   - **스마트폰 부문**에서는 Galaxy S23, S23+, S23 Ultra가 주력 제품이었습니다【5:0†source】.
   - **TV 부문**에서는 QLED TV와 OLED TV에서 중요한 발전이 있었으며, 특히 Flat QLED 4K TV 제품군이 주력이었습니다【5:1†source】.
   - **노트북 부문**에서는 Galaxy Book3 Ultra와 관련 제품이 주목할 만한 신제품이었습니다【5:3†source】.

2. **LG전자**: 2023년 주력 상품은 무선 OLED TV였습니다. 이 제품은 공간 활용성을 높이고 고객 경험 혁신을 목표로 설계되었으며, 다양한 사이즈와 OLED evo 라인업 확대를 통한 프리미엄 시장 주도를 목표로 했습니다【5:10†source】.

3. **Apple Inc.**: Apple의 2023년 주력 제품은 iPhone이었습니다. 2023년 Apple iPhone의 매출은 전년 대비 2% 감소했지만 여전히 Apple의 가장 큰 매출원이었습니다【5:16†source】.

4. **NVIDIA**: 2023년 NVIDIA의 주력 제품은 GeForce RTX 4060 및 4070 GPUs였습니다. 이 GPU들은 NVIDIA Ada Lovelace 아키텍처를 기반으로 하며, 게임의 AI 기반 자연어 상호작용을 변화시키는 데 중점을 두었습니다【5:17†source】.

annotations >>

【5:0†source】 [삼성전자]사업보고서(2024.03.12).pdf 130 142
【5:1†source】 [삼성전자]사업보고서(2024.03.12).pdf 226 238
【5:3†source】 [삼성전자]사업보고서(2024.03.12).pdf 301 313
【5:10†source】 [LG전자]

In [ ]:
# 3. 사용자 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="데이터의 각 회사가 발행한 주식의 수를 비교하시오."
)

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="각 회사의 2023년 주력 상품은 무엇이었나요?"
)

# 4. 런 생성 및 결과 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# 5. 런 상태 확인 및 런 스탭 내용 확인
if "completed" == run.status:
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread.id,
        run_id=run.id,
        order="asc"
    )
    
    df_run_steps = pd.DataFrame([data.__dict__ for data in run_steps.data])
    for details in df_run_steps["step_details"]:
        if "tool_calls" == details.type:
            for tool_call in details.tool_calls:
                print(f"tool called : {tool_call.type}, {tool_call.id}")
                
        elif "message_creation" == details.type:
            msgid = details.message_creation.message_id
            msg = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=msgid
            )

            for content in msg.content:
                print(content.text.value)
                print("\nannotations >>\n")
                for annot in content.text.annotations:
                    file = client.files.retrieve(file_id=annot.file_citation.file_id)
                    print(annot.text, file.filename, annot.start_index, annot.end_index)
else:
    print(run.status)

In [ ]:
# 6. 어시스턴트 삭제
res = client.beta.assistants.delete(
    assistant_id=assistant.id
)
print(f"assistant deleted: {res.deleted}")

# 7. 스레드 삭제
res = client.beta.threads.delete(
    thread_id=thread.id
)
print(f"thread deleted: {res.deleted}")


In [ ]:

# 2. 어시스턴트 생성; 파일 검색 도구 추가 
assistant = client.beta.assistants.create(
    name="재무 분석 전문가",
    instructions="""
        당신은 재무 분석 전문가입니다. 제공한 사업보고서 및 재무제표 정보를 바탕으로 질문에 답하시오.
        """,
    model=OPENAI_MODEL,
    tools=[{"type": "file_search"}],
    tool_resources={     
        "file_search": {
            "vector_store_ids": [vector_store.id]
        }
    }
)

# 3. 사용자 메시지 생성
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="데이터의 각 회사가 발행한 주식의 수를 비교하시오."
)

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="각 회사의 2023년 주력 상품은 무엇이었나요?"
)
  
# 4. 런 생성 및 결과 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# 5. 런 상태 확인 및 런 스탭 내용 확인
if "completed" == run.status:
    run_steps = client.beta.threads.runs.steps.list(
        thread_id=thread.id,
        run_id=run.id,
        order="asc"
    )
    
    df_run_steps = pd.DataFrame([data.__dict__ for data in run_steps.data])
    for details in df_run_steps["step_details"]:
        if "tool_calls" == details.type:
            for tool_call in details.tool_calls:
                print(f"tool called : {tool_call.type}, {tool_call.id}")
                
        elif "message_creation" == details.type:
            msgid = details.message_creation.message_id
            msg = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=msgid
            )

            for content in msg.content:
                print(content.text.value)
                print("\nannotations >>\n")
                for annot in content.text.annotations:
                    file = client.files.retrieve(file_id=annot.file_citation.file_id)
                    print(annot.text, file.filename, annot.start_index, annot.end_index)
else:
    print(run.status)

# 6. 어시스턴트 삭제
res = client.beta.assistants.delete(
    assistant_id=assistant.id
)
print(f"assistant deleted: {res.deleted}")

# 7. 스레드 삭제
res = client.beta.threads.delete(
    thread_id=thread.id
)
print(f"thread deleted: {res.deleted}")
